In [1]:
# Include necessary imports
import os
import torch 
import pandas as pd
from torch.utils.data import DataLoader
from music21 import *
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler



In [16]:
# Preprocess the data

folder_path = 'Data/'
test = []
train = []
validation = []
for dirname in os.listdir(folder_path):
    if dirname != '.DS_Store':
        for filename in os.listdir(folder_path + dirname):
            if filename != '.ipynb_checkpoints':
                df = pd.read_csv(folder_path + dirname + '/' + filename)
                transposed_df = df.transpose()
                if dirname == 'test':
                    test.append(transposed_df)
                if dirname == 'train':
                    print(filename)
                    train.append(transposed_df)
                if dirname == 'valid':
                    validation.append(transposed_df)

chorale_198.csv
chorale_173.csv
chorale_167.csv
chorale_007.csv
chorale_013.csv
chorale_239.csv
chorale_205.csv
chorale_211.csv
chorale_210.csv
chorale_204.csv
chorale_238.csv
chorale_012.csv
chorale_006.csv
chorale_166.csv
chorale_172.csv
chorale_199.csv
chorale_158.csv
chorale_164.csv
chorale_170.csv
chorale_038.csv
chorale_010.csv
chorale_004.csv
chorale_212.csv
chorale_206.csv
chorale_207.csv
chorale_213.csv
chorale_005.csv
chorale_011.csv
chorale_039.csv
chorale_171.csv
chorale_165.csv
chorale_159.csv
chorale_161.csv
chorale_175.csv
chorale_149.csv
chorale_015.csv
chorale_001.csv
chorale_029.csv
chorale_217.csv
chorale_203.csv
chorale_202.csv
chorale_216.csv
chorale_028.csv
chorale_000.csv
chorale_014.csv
chorale_148.csv
chorale_174.csv
chorale_160.csv
chorale_189.csv
chorale_176.csv
chorale_162.csv
chorale_002.csv
chorale_016.csv
chorale_200.csv
chorale_214.csv
chorale_228.csv
chorale_229.csv
chorale_215.csv
chorale_201.csv
chorale_017.csv
chorale_003.csv
chorale_163.csv
chorale_

# Model

In [3]:
class Model(torch.nn.Module):
    def __init__(self, input_size, output_size, hidden_dim=40, n_layers=3):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.lstm = torch.nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_size)
        
    def forward(self, x, hidden=None):
        lstm_output, (h,c) = self.lstm(x, hidden)
        model_output = self.fc(lstm_output)
        return model_output


# Train

In [4]:
def train_model(model, melody, harmonies, optimizer, criterion, num_epochs):
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        output = model(melody)
        loss = criterion(output, harmonies)
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 100 == 0:
            print("Epoch: ", epoch, "Loss: ", loss)
        model.eval()

In [5]:
# Min-Max normalization technique
def normalize_array(array):
    X_std = (array - 1) / (88 - 1)
    return X_std

criterion = torch.nn.MSELoss()
epoch = 5000
# index = 0
# for song in train:
#     index = index + 1
# print("training song ", index)
song = train[0] # REMOVE
melody = torch.tensor(normalize_array(song.iloc[0].values.reshape(-1,1)), dtype=torch.float32).unsqueeze(0).reshape(1,song.shape[1],1)
harmonies = torch.tensor(normalize_array(song.iloc[1:].values.T), dtype=torch.float32).unsqueeze(0)
model = Model(1, harmonies.shape[2])
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
train_model(model, melody, harmonies, optimizer, criterion, epoch)

Epoch:  99 Loss:  tensor(0.0020, grad_fn=<MseLossBackward0>)
Epoch:  199 Loss:  tensor(0.0016, grad_fn=<MseLossBackward0>)
Epoch:  299 Loss:  tensor(0.0014, grad_fn=<MseLossBackward0>)
Epoch:  399 Loss:  tensor(0.0014, grad_fn=<MseLossBackward0>)
Epoch:  499 Loss:  tensor(0.0014, grad_fn=<MseLossBackward0>)
Epoch:  599 Loss:  tensor(0.0013, grad_fn=<MseLossBackward0>)
Epoch:  699 Loss:  tensor(0.0013, grad_fn=<MseLossBackward0>)
Epoch:  799 Loss:  tensor(0.0013, grad_fn=<MseLossBackward0>)
Epoch:  899 Loss:  tensor(0.0014, grad_fn=<MseLossBackward0>)
Epoch:  999 Loss:  tensor(0.0014, grad_fn=<MseLossBackward0>)
Epoch:  1099 Loss:  tensor(0.0014, grad_fn=<MseLossBackward0>)
Epoch:  1199 Loss:  tensor(0.0014, grad_fn=<MseLossBackward0>)
Epoch:  1299 Loss:  tensor(0.0013, grad_fn=<MseLossBackward0>)
Epoch:  1399 Loss:  tensor(0.0013, grad_fn=<MseLossBackward0>)
Epoch:  1499 Loss:  tensor(0.0014, grad_fn=<MseLossBackward0>)
Epoch:  1599 Loss:  tensor(0.0013, grad_fn=<MseLossBackward0>)
Epo

# Hyperparameter Tuning

In [ ]:
learning = [0.01]
n_layers= [1,2,3]
hidden_dim = [20, 40, 50]
epochs= [5000]
best_loss = float('inf')
best_params = {}


for LR in learning:
    for n_layer in n_layers:
        for epoch in epochs:
            for dims in hidden_dim:
                print(f"Training with LR={LR} and n_layers={n_layer} and epochs={epoch} and hidden_dims={dims}")
                model = Model(input_size=1, output_size=harmonies.shape[2], n_layers=n_layer, hidden_dim=dims)
                optimizer = torch.optim.Adam(model.parameters(), lr=LR)
                train_model(model, melody, harmonies, optimizer, criterion, epoch)
                with torch.no_grad():
                    output = model(melody)
                    loss = criterion(output, harmonies)
                    print(f"Final Loss: {loss.item()}")        
                # Keep track of the best model (with lowest loss)
                if loss.item() < best_loss:
                    best_loss = loss.item()
                    best_params = {'learning': LR, 'n_layers': n_layer, 'epochs': epoch, 'hidden_dim': dims}
print("BEST: ", best_params)

In [ ]:
# tiny = pd.DataFrame([[67,62,59,43], [68,62,59,43]]).transpose()
# melody = torch.tensor(tiny.iloc[0], dtype=torch.float32).unsqueeze(0).reshape(1,2,1)
# harmonies = torch.transpose(torch.tensor(tiny.iloc[1:].values, dtype=torch.float32),0,1).unsqueeze(0)
# model = Model(1, harmonies.shape[2])
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# criterion = torch.nn.MSELoss()
# train_model(model, melody, harmonies, optimizer, criterion, 10000)

In [35]:
def inverse_array(array):
    X_scaled = array * (88 - 1) + 1
    return X_scaled

def midi_to_note(part):
    print(type(part))
    result = stream.Part()
    count = 1
    prev = part[0]
    for i in range(1, len(part)):
        pitch = part[i]
        curr = round(pitch)
        if curr == prev:
            count += 1
        else:
            this_note = note.Note(prev, quarterLength=count/4)
            print(count / 4)
            result.append(this_note)
            count = 1
        prev = curr
    return result

def output_to_sheet_music(melody, result):
    result_numpy = result.detach().numpy()
    inversed = inverse_array(np.squeeze(result_numpy)).T
    
    score = stream.Score()
    melody_part = midi_to_note(melody)
    
    alto_notes = inversed[0]
    tenor_notes = inversed[1]
    bass_notes = inversed[2]  
    
    alto_part = midi_to_note(alto_notes)
    tenor_part = midi_to_note(tenor_notes)
    bass_part = midi_to_note(bass_notes)

#     for pitch in alto_notes:
#         alto_note = note.Note(round(pitch.item()), quarterLength=0.25)
#         alto_part.append(alto_note)
#     for pitch in tenor_notes:
#         tenor_note = note.Note(round(pitch.item()), quarterLength=0.25)
#         tenor_part.append(tenor_note)
#     for pitch in bass_notes:
#         bass_note = note.Note(round(pitch.item()), quarterLength=0.25)
#         bass_part.append(bass_note)
    score.append(melody_part)
    score.append(alto_part)
    score.append(tenor_part)
    score.append(bass_part)
    score.show('midi')
    score.write('musicxml', 'output.xml')

In [37]:
melody = train[0].iloc[0]
# print("melody is: ", torch.tensor(normalize_array(song.iloc[0].values.reshape(-1,1)), dtype=torch.float32).unsqueeze(0).reshape(1,song.shape[1],1))
harmonies = torch.tensor(normalize_array(song.iloc[1:].values.T), dtype=torch.float32).unsqueeze(0)
result = model(torch.tensor(normalize_array(train[0].iloc[0].values.reshape(-1,1)), dtype=torch.float32).unsqueeze(0).reshape(1,train[0].shape[1],1))
output_to_sheet_music(melody, harmonies)

<class 'pandas.core.series.Series'>
0.5
0.5
1.0
1.0
1.5
0.5
1.0
1.0
1.0
1.0
1.0
1.0
2.0
2.0
2.0
2.0
1.0
1.0
2.0
1.0
1.0
1.0
0.5
1.5
1.0
1.0
1.0
2.0
2.0
2.0
2.0
1.0
0.5
2.5
<class 'numpy.ndarray'>
1.0
1.0
1.0
3.0
1.0
1.0
1.5
0.5
1.0
2.0
2.5
0.25
0.25
1.5
0.5
0.5
0.5
1.5
1.5
1.0
1.0
1.5
0.5
1.0
1.0
0.5
0.5
0.5
0.5
0.5
0.5
1.0
0.5
0.5
1.0
0.5
0.5
1.0
0.5
0.5
0.5
0.5
1.5
0.5
1.5
0.25
0.25
1.0
<class 'numpy.ndarray'>
0.25
0.25
0.5
1.5
0.5
1.0
1.0
1.5
0.5
2.0
1.0
4.0
1.5
0.25
0.25
0.5
0.5
1.0
0.5
0.5
0.5
1.0
1.0
0.5
0.5
0.5
2.0
0.5
0.5
1.0
2.0
1.0
2.0
1.5
0.5
1.5
0.5
1.5
0.5
0.5
0.5
0.5
0.5
1.0
0.5
0.5
0.5
0.5
<class 'numpy.ndarray'>
1.25
0.25
0.25
0.25
0.5
0.5
2.0
0.5
0.5
1.0
1.0
1.0
1.0
0.5
0.5
1.0
1.0
0.5
0.5
0.5
0.5
0.5
0.5
1.0
0.5
0.5
0.5
1.0
0.5
0.5
0.5
0.5
0.5
1.0
1.0
0.5
0.5
0.5
0.5
1.0
1.0
1.0
0.5
0.5
0.5
0.5
1.0
0.5
0.5
1.5
0.5
1.5
0.5
1.0
0.5
0.5
1.0
0.5
0.5
1.0
1.0


In [ ]:
# Finetune (hyperparameters, move around test data (refer to notes), etc)

In [ ]:
# Test with new data + evaluate

In [ ]:
# Make any other changes

In [ ]:
# Sheet music + audio (musicAI)

In [ ]:
# Create new models if time permits (follow steps 3 - 7)

In [ ]:
# Compare models

In [ ]:
# Front end ** if time permits
# - Interactive sheet music
# - musescore front end??

In [7]:
# real one and generated compare
# train on all songs + test on a different song
# measure the test loss not just the training loss
